In [1]:
import glob
import os
import numpy as np
import errno

In [2]:
def force_symlink(file1, file2):
    try:
        os.symlink(file1, file2)
    except OSError, e:
        if e.errno == errno.EEXIST:
            os.remove(file2)
            os.symlink(file1, file2)

In [3]:
!jupyter nbextension enable --py --sys-prefix widgetsnbextension
def log_progress(sequence, title='', every=1, size=None, end_msg='', removeAtTheEnd=False):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{index} / ?'.format(index=index)
                else:
                    progress.value = index
                    if isinstance(sequence, dict):
                        val = ''
                    else:
                        val=sequence[index-1]
                            
                    label.value = u'{title} : {value} ({index} / {size})'.format(
                        title=title,
                        value=val,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = end_msg
        if removeAtTheEnd:
            box.close()
    

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [4]:
pathBasic='/home/jguerry/workspace/datasets/ONERA.ROOM'
allFolders=sorted(glob.glob(pathBasic+"/RGBD/*"))

In [20]:
for pathFolder in allFolders:
    folder = os.path.split(pathFolder)[-1]
    print 40*'-'
    print folder
    if '*' in folder:
        print 'pass'
        continue
    if '2017' not in folder:
        print 'pass'
        continue
    print 'Globing...'
    imgs_rgb=sorted(glob.glob(pathFolder+"/image/*"))
    imgs_depth=sorted(glob.glob(pathFolder+"/depth_8bits/*"))
    annot_xml=sorted(glob.glob(pathFolder+"/bb/voc/*"))
    print 'Globbed'
    
    print "Associating..."
    array_rgb2depth_ts = np.zeros(len(imgs_rgb),dtype=np.int)
    for id_img_rgb,filename_rgb in enumerate(imgs_rgb):
        ts_rgb = int(filename_rgb.split("-")[-1].split(".")[0])
        id_img_depth_closest = 0
        ts_depth_closest = 0
        for id_img_depth,filename_depth in enumerate(imgs_depth):
            ts_depth = int(filename_depth.split("-")[-1].split(".")[0])
            if abs(ts_rgb-ts_depth)<=abs(ts_rgb-ts_depth_closest):
                ts_depth_closest = ts_depth
                id_img_depth_closest = id_img_depth
        array_rgb2depth_ts[id_img_rgb]=id_img_depth_closest
    print "Associated"
    
    print 'Symlinking...'
    for i in range(len(imgs_rgb)):
        
        
        fileName_rgb = os.path.split(os.path.split(imgs_rgb[i])[-1])[-1]
        newPathRGB = pathBasic+'/data_pv/rgb/'+folder+'_'+fileName_rgb
        force_symlink(imgs_rgb[i], newPathRGB)
        
        
        fileName_depth = os.path.split(os.path.split(imgs_depth[array_rgb2depth_ts[i]])[-1])[-1]
        newPathDepth = pathBasic+'/data_pv/depth_8bits/'+folder+'_'+fileName_rgb[:-4]+'.png'
        #force_symlink(imgs_depth[array_rgb2depth_ts[i]], newPathDepth)
        
        fileName_xml = os.path.split(os.path.split(annot_xml[i])[-1])[-1]
        newPathxml = pathBasic+'/data_pv/annotations/'+folder+'_'+fileName_xml
        force_symlink(annot_xml[i], newPathxml)
    print folder, len(imgs_rgb), 'done.'
print 'Finished.'    

----------------------------------------
2015-09-18-16-08-12
pass
----------------------------------------
2015-10-05-15-45-29*
pass
----------------------------------------
2015-11-19-14-55-21
pass
----------------------------------------
2015-11-19-15-01-05
pass
----------------------------------------
2015-11-25-11-24-37*
pass
----------------------------------------
2015-11-25-11-27-15*
pass
----------------------------------------
2015-11-25-11-29-15*
pass
----------------------------------------
2016-02-02-14-58-24
pass
----------------------------------------
2016-02-05-16-06-34*
pass
----------------------------------------
2016-02-05-16-08-04
pass
----------------------------------------
2016-02-05-16-09-36
pass
----------------------------------------
2016-02-05-16-10-42*
pass
----------------------------------------
2016-02-05-16-12-11
pass
----------------------------------------
2016-02-05-16-14-59
pass
----------------------------------------
2016-02-05-16-17-46
pass
----

In [7]:
sequences = {'easy':['2015-09-18-16-08-12','2016-02-05-16-08-04','2016-02-05-16-09-36'],
'average':['2015-11-19-14-55-21','2016-02-05-16-17-46','2016-02-02-14-58-24','2015-11-19-15-01-05','2016-02-05-16-14-59','2016-02-05-16-19-48','2016-04-28-17-46-13'],
'hard':['2016-04-28-17-50-10' ,'2016-04-28-17-42-48','2016-04-28-17-45-25','2016-02-05-16-18-49','2016-02-05-16-12-11'],
'or_2017':['2017-04-26-14-06-49','2017-04-26-14-33-16','2017-04-26-14-40-34','2017-04-26-14-42-11','2017-04-26-14-50-49','2017-04-26-15-09-53','2017-04-26-15-12-54','2017-04-26-15-45-24','2017-04-26-15-47-03'],
'static':['2017-04-26-14-06-49'],
'sar':['2016-04-28-17-50-10','2016-04-28-17-45-25','2016-02-05-16-18-49', '2016-02-05-16-12-11', '2017-04-26-14-50-49'],
'2017_parking':['2017-04-26-15-45-24','2017-04-26-15-47-03']}



In [8]:
for seq,folders in log_progress(sequences.iteritems(),title='Step', end_msg='Finished !'): 
    print seq,len(folders)
        
    f = open('/home/jguerry/workspace/datasets/ONERA.ROOM/data_pv/sets/'+seq+'.txt', 'w')
    for pathFolder in log_progress(folders,title='Step',removeAtTheEnd=True): 
        folder='/home/jguerry/workspace/datasets/ONERA.ROOM/RGBD/'+pathFolder
# allFolders=sorted(glob.glob(pathBasic+"/RGBD/*"))
        print folder
#         pass
#         folder = os.path.split(pathFolder)[-1]
        #         print 40*'-'
        #         print folder
        if pathFolder not in folders:
            #             print 'pass'
            continue
        
        #         print 'Globing...'
        imgs_rgb=sorted(glob.glob(folder+"/image/*"))
        #         print 'Globbed'

        #print 'Printing set...'
        for i in range(len(imgs_rgb)):  
            fileName_rgb = os.path.split(os.path.split(imgs_rgb[i])[-1])[-1]
            newfileName_rgb = pathFolder+'_'+fileName_rgb[:-4]
            f.write(newfileName_rgb+'\n')
        print folder, len(imgs_rgb), 'done.'
    #print 'Finished.'
    f.close()

average 7
/home/jguerry/workspace/datasets/ONERA.ROOM/RGBD/2015-11-19-14-55-21
/home/jguerry/workspace/datasets/ONERA.ROOM/RGBD/2015-11-19-14-55-21 1504 done.
/home/jguerry/workspace/datasets/ONERA.ROOM/RGBD/2016-02-05-16-17-46
/home/jguerry/workspace/datasets/ONERA.ROOM/RGBD/2016-02-05-16-17-46 463 done.
/home/jguerry/workspace/datasets/ONERA.ROOM/RGBD/2016-02-02-14-58-24
/home/jguerry/workspace/datasets/ONERA.ROOM/RGBD/2016-02-02-14-58-24 943 done.
/home/jguerry/workspace/datasets/ONERA.ROOM/RGBD/2015-11-19-15-01-05
/home/jguerry/workspace/datasets/ONERA.ROOM/RGBD/2015-11-19-15-01-05 2190 done.
/home/jguerry/workspace/datasets/ONERA.ROOM/RGBD/2016-02-05-16-14-59
/home/jguerry/workspace/datasets/ONERA.ROOM/RGBD/2016-02-05-16-14-59 1568 done.
/home/jguerry/workspace/datasets/ONERA.ROOM/RGBD/2016-02-05-16-19-48
/home/jguerry/workspace/datasets/ONERA.ROOM/RGBD/2016-02-05-16-19-48 1052 done.
/home/jguerry/workspace/datasets/ONERA.ROOM/RGBD/2016-04-28-17-46-13
/home/jguerry/workspace/datas

In [10]:
# code to generate set file no2017
f = open('/home/jguerry/workspace/datasets/ONERA.ROOM/data_pv/sets/2017_no_parking.txt', 'w')
for pathFolder in allFolders:
    folder = os.path.split(pathFolder)[-1]
    if '*' in folder:
#         print 'pass'
        continue
    if '2017' not in folder:
#         print 'pass'
        continue
        
    if folder in ['2017-04-26-15-45-24','2017-04-26-15-47-03']:
        print 'pass'
        continue
        
        
        
    print 40*'-'
    print folder
    print 'Globing...'
    imgs_rgb=sorted(glob.glob(pathFolder+"/image/*"))
    print 'Globbed'
    
    print 'Printing set...'
    for i in range(len(imgs_rgb)):  
        fileName_rgb = os.path.split(os.path.split(imgs_rgb[i])[-1])[-1]
        newfileName_rgb = folder+'_'+fileName_rgb[:-4]
        f.write(newfileName_rgb+'\n')
    print folder, len(imgs_rgb), 'done.'
print 'Finished.'
f.close()

----------------------------------------
2017-04-26-14-06-49
Globing...
Globbed
Printing set...
2017-04-26-14-06-49 1980 done.
----------------------------------------
2017-04-26-14-33-16
Globing...
Globbed
Printing set...
2017-04-26-14-33-16 560 done.
----------------------------------------
2017-04-26-14-42-11
Globing...
Globbed
Printing set...
2017-04-26-14-42-11 424 done.
----------------------------------------
2017-04-26-14-50-49
Globing...
Globbed
Printing set...
2017-04-26-14-50-49 950 done.
----------------------------------------
2017-04-26-15-09-53
Globing...
Globbed
Printing set...
2017-04-26-15-09-53 1910 done.
----------------------------------------
2017-04-26-15-12-54
Globing...
Globbed
Printing set...
2017-04-26-15-12-54 1532 done.
pass
pass
Finished.


In [30]:
# code to generate set file ALL
f = open('/home/jguerry/workspace/datasets/ONERA.ROOM/data_pv/sets/all.txt', 'w')
for pathFolder in allFolders:
    folder = os.path.split(pathFolder)[-1]
    print 40*'-'
    print folder
    if '*' in folder:
        print 'pass'
        continue
    print 'Globing...'
    imgs_rgb=sorted(glob.glob(pathFolder+"/image/*"))
    print 'Globbed'
    
    print 'Printing set...'
    for i in range(len(imgs_rgb)):  
        fileName_rgb = os.path.split(os.path.split(imgs_rgb[i])[-1])[-1]
        newfileName_rgb = folder+'_'+fileName_rgb[:-4]
        f.write(newfileName_rgb+'\n')
    print folder, len(imgs_rgb), 'done.'
print 'Finished.'
f.close()

----------------------------------------
2015-09-18-16-08-12
Globing...
Globbed
Printing set...
2015-09-18-16-08-12 2908 done.
----------------------------------------
2015-10-05-15-45-29*
pass
----------------------------------------
2015-11-19-14-55-21
Globing...
Globbed
Printing set...
2015-11-19-14-55-21 1504 done.
----------------------------------------
2015-11-19-15-01-05
Globing...
Globbed
Printing set...
2015-11-19-15-01-05 2190 done.
----------------------------------------
2015-11-25-11-24-37*
pass
----------------------------------------
2015-11-25-11-27-15*
pass
----------------------------------------
2015-11-25-11-29-15*
pass
----------------------------------------
2016-02-02-14-58-24
Globing...
Globbed
Printing set...
2016-02-02-14-58-24 943 done.
----------------------------------------
2016-02-05-16-06-34*
pass
----------------------------------------
2016-02-05-16-08-04
Globing...
Globbed
Printing set...
2016-02-05-16-08-04 2030 done.
-------------------------------

In [ ]:
todo : refaire tous les sets (vu que y'a des nouveaux noms de dossiers')
    Faut il enlever les étoiles ?
    faire les nouveaux sets 2017 (easy, average, hard)
    faire les sets nb, static, couchés